In [ ]:
from gurobipy import Model, GRB, quicksum
import pandas as pd
import numpy as np
from itertools import product

# ========= Load data =========
tasks = pd.read_csv("./open_v2/task.csv")
agvs  = pd.read_csv("./open_v2/agv.csv")

# ========= Sets & data =========
depot_xy = (0.0, 0.0)

# Node indexing: 0 = start depot, 1..n = tasks, n+1 = end depot
n_tasks = len(tasks)
nodes = list(range(n_tasks + 2))
START, END = 0, n_tasks + 1
N = list(range(1, n_tasks + 1))         # task nodes only
N_st = nodes                              # start+tasks+end
N_s = list(range(0, n_tasks + 1))        # allowed predecessors (not END)
N_t = list(range(1, n_tasks + 2))        # allowed successors (not START)

# coordinates aligned with nodes
xs = [depot_xy[0]] + tasks["x"].tolist() + [depot_xy[0]]
ys = [depot_xy[1]] + tasks["y"].tolist() + [depot_xy[1]]

# Distance matrix (Manhattan)
xs_arr = np.array(xs, dtype=float)
ys_arr = np.array(ys, dtype=float)
distance = np.abs(xs_arr[:, None] - xs_arr[None, :]) + np.abs(ys_arr[:, None] - ys_arr[None, :])
distance = distance.tolist()

# Sets of AGVs and route limits
K = list(range(len(agvs)))
Rk = {k: list(range(4)) for k in K}  # or use per-AGV T_max if desired

# Parameters
v = agvs['speed_cells_per_sec'].tolist()       # speed per AGV
Q = agvs['capacity'].tolist()                  # capacity per AGV
Dmax = agvs['max_distance'].tolist()           # max distance per route
s = [0.0] + tasks['service_time'].astype(float).tolist() + [0.0]
u = [0.0] + tasks['demand'].astype(float).tolist() + [0.0]
dead = [0.0] + tasks['deadline'].astype(float).tolist() + [0.0]

alpha, beta = 1.0, 3.0

# Precompute travel times per AGV: tt[k][i][j] = distance[i][j] / v[k]
tt = {k: [[distance[i][j] / max(1e-9, v[k]) for j in N_st] for i in N_st] for k in K}

# Arc set A = {(i,j) | i != j} but avoid illegal arcs involving START/END
A = [(i, j) for i in N_s for j in N_t if i != j]  # excludes END->*, *->START automatically

# Conservative Big-M per (k,i,j): deadline span + travel + service
# If deadlines are 0 for depots, span ≈ max(dead) - min(dead) among tasks
if len(N) > 0:
    dmin = float(np.min([dead[i] for i in N]))
    dmax = float(np.max([dead[i] for i in N]))
else:
    dmin, dmax = 0.0, 0.0
deadline_span = max(0.0, dmax - dmin)
M_ij = {(k, i, j): (deadline_span + tt[k][i][j] + s[i] + s[j] + 1.0) for k in K for (i, j) in A}

# ========= Build model =========
m = Model("AGV_Routing_Fast")

# Variables
# x[k,r,i,j] in {0,1}; y[k,r,i] in {0,1}; o[k,r,i] >=0 integer; t[k,r,i] >=0; late[k,r,i] >=0
x = m.addVars(
    [(k, r, i, j) for k in K for r in Rk[k] for (i, j) in A],
    vtype=GRB.BINARY, name="x"
)
y = m.addVars(
    [(k, r, i) for k in K for r in Rk[k] for i in N],
    vtype=GRB.BINARY, name="y"
)
o = m.addVars(
    [(k, r, i) for k in K for r in Rk[k] for i in N],
    vtype=GRB.INTEGER, lb=0, name="o"
)
t = m.addVars(
    [(k, r, i) for k in K for r in Rk[k] for i in N_st],
    vtype=GRB.CONTINUOUS, lb=0.0, name="t"
)
late = m.addVars(
    [(k, r, i) for k in K for r in Rk[k] for i in N],
    vtype=GRB.CONTINUOUS, lb=0.0, name="late"
)

m.update()

# ========= Constraints =========

# 1) Route starts: each AGV has at most one departure from START per route, and exactly one at r=0
for k in K:
    # r = 0 must leave START once
    m.addConstr(quicksum(x[k, 0, START, j] for j in N) == 1, name=f"start_once_k{k}_r0")
    # subsequent routes optional
    for r in Rk[k]:
        if r == 0:
            continue
        m.addConstr(quicksum(x[k, r, START, j] for j in N) <= 1, name=f"start_opt_k{k}_r{r}")

# 2) Flow conservation & visit linking (for task nodes only)
#    inflow == outflow, and inflow == y
for k in K:
    for r in Rk[k]:
        for j in N:
            inflow = quicksum(x[k, r, i, j] for i in N_s if i != j)
            outflow = quicksum(x[k, r, j, h] for h in N_t if h != j)
            m.addConstr(inflow == outflow, name=f"flow_k{k}_r{r}_j{j}")
            m.addConstr(inflow == y[k, r, j], name=f"visitlink_k{k}_r{r}_j{j}")

# 3) Time sequencing with tightened Big-M (for all arcs in A)
for k in K:
    for r in Rk[k]:
        for (i, j) in A:
            # t_j >= t_i + service_i + travel(i,j) - M*(1 - x_ij)
            m.addConstr(
                t[k, r, j] >= t[k, r, i] + s[i] + tt[k][i][j] - M_ij[k, i, j] * (1 - x[k, r, i, j]),
                name=f"time_k{k}_r{r}_{i}_{j}"
            )

# 4) Served amount only if visited (here: full demand if visited; no split)
for k in K:
    for r in Rk[k]:
        for i in N:
            m.addConstr(o[k, r, i] == u[i] * y[k, r, i], name=f"serveonlyifvisit_k{k}_r{r}_i{i}")

# 5) Capacity per route
for k in K:
    for r in Rk[k]:
        m.addConstr(quicksum(o[k, r, i] for i in N) <= Q[k], name=f"cap_k{k}_r{r}")

# 6) Max distance per route
for k in K:
    for r in Rk[k]:
        m.addConstr(
            quicksum(distance[i][j] * x[k, r, i, j] for (i, j) in A) <= Dmax[k],
            name=f"dist_k{k}_r{r}"
        )

# 7) Each task served exactly once (no split across routes/vehicles)
for j in N:
    m.addConstr(quicksum(y[k, r, j] for k in K for r in Rk[k]) == 1, name=f"once_j{j}")

# 8) Start times
for k in K:
    # first route starts at 0 at START
    m.addConstr(t[k, Rk[k][0], START] == 0.0, name=f"t0_k{k}")
    # route chaining: END of r <= START time of r+1  (we approximate END time by the last arrival back to END)
    # To enforce reaching END when a route is used, add virtual arrival:
    # If you want strict return-to-depot, uncomment the next two blocks.

    # Ensure arriving to END if route used
    for r in Rk[k]:
        m.addConstr(
            quicksum(x[k, r, i, END] for i in N_s if i != END) ==
            quicksum(x[k, r, START, j] for j in N), name=f"end_used_iff_start_k{k}_r{r}"
        )

    # Time of END >= arrivals into END
    for r in Rk[k]:
        for i in N_s:
            if i == END: 
                continue
            m.addConstr(
                t[k, r, END] >= t[k, r, i] + s[i] + tt[k][i][END] - M_ij[k, i, END] * (1 - x[k, r, i, END]),
                name=f"time_to_end_k{k}_r{r}_i{i}"
            )

    # Route sequencing: END time of r <= START time of r+1
    for r_idx in range(len(Rk[k]) - 1):
        r, rnext = Rk[k][r_idx], Rk[k][r_idx + 1]
        m.addConstr(t[k, r, END] <= t[k, rnext, START], name=f"chain_k{k}_r{r}_to_{rnext}")

# 9) Tardiness w.r.t. deadlines, with activation by y
for k in K:
    for r in Rk[k]:
        for i in N:
            m.addConstr(late[k, r, i] >= t[k, r, i] - dead[i], name=f"late_lb_k{k}_r{r}_i{i}")
            m.addConstr(late[k, r, i] <= (deadline_span + max(dead)) * y[k, r, i], name=f"late_act_k{k}_r{r}_i{i}")

# ========= Objective =========
travel_cost = quicksum(distance[i][j] * x[k, r, i, j] for k in K for r in Rk[k] for (i, j) in A)
tardiness   = quicksum(late[k, r, i] for k in K for r in Rk[k] for i in N)

m.setObjective(alpha * travel_cost + beta * tardiness, GRB.MINIMIZE)

# ========= Solver params (balanced for faster optimality) =========
m.Params.OutputFlag   = 1
m.Params.MIPFocus     = 1       # lean to finding high-quality incumbents quickly
m.Params.Heuristics   = 0.2     # moderate primal heuristics
m.Params.Presolve     = 2       # aggressive presolve
m.Params.Cuts         = 1       # moderate cuts
m.Params.TimeLimit  = 3600     # (optional) keep off if you want proven optimality
# m.Params.Threads    = 0       # let Gurobi decide; set to a fixed number if you want reproducibility

m.optimize()


Set parameter OutputFlag to value 1
Set parameter MIPFocus to value 1
Set parameter Heuristics to value 0.2
Set parameter Presolve to value 2
Set parameter Cuts to value 1
Set parameter TimeLimit to value 3600
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) i7-14700, instruction set [SSE2|AVX|AVX2]
Thread count: 20 physical cores, 28 logical processors, using up to 28 threads

Non-default parameters:
TimeLimit  3600
Heuristics  0.2
MIPFocus  1
Cuts  1
Presolve  2

Optimize a model with 642520 rows, 630180 columns and 4314525 nonzeros
Model fingerprint: 0x7876416f
Variable types: 12120 continuous, 618060 integer (612060 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [1e+00, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 9e+02]
Presolve removed 13650 rows and 6075 columns (presolve time = 5s)...
Presolve removed 13650 rows and 6090 columns (presolve time = 10s)...
Pr

In [ ]:
# === AGV 별 전체 경로 문자열로 저장 ===
from collections import defaultdict

agv_routes = defaultdict(dict)  # agv_routes[k][r] = [(i,j)]

if model.status in [GRB.OPTIMAL, GRB.TIME_LIMIT, GRB.INTERRUPTED]:
    # 1) 아크 수집
    for (k, i, j, r), var in x.items():
        if var.X > 0.5:
            agv_routes[k].setdefault(r, []).append((i, j))

    final_paths = []
    for k, routes in agv_routes.items():
        full_path = []
        for r in sorted(routes.keys()):
            arcs = routes[r]
            if not arcs:
                continue
            # 2) 순서대로 경로 복원
            route = []
            current = 0  # 항상 DEPOT에서 시작
            used = set()
            while True:
                next_nodes = [j for (i, j) in arcs if i == current and (i, j) not in used]
                if not next_nodes:
                    break
                nxt = next_nodes[0]
                route.append(nxt)
                used.add((current, nxt))
                current = nxt

            # 3) 포맷팅: 0/end → DEPOT, 나머지 → Txxxx
            formatted = ["DEPOT"]
            for node in route:
                if node in (0, len(N_s_t) - 1):
                    formatted.append("DEPOT")
                else:
                    formatted.append(f"T{node:04d}")
            full_path.extend(formatted)

        # 4) 중복 DEPOT 제거
        clean_path = [full_path[0]]
        for node in full_path[1:]:
            if node != "DEPOT" or clean_path[-1] != "DEPOT":
                clean_path.append(node)

        final_paths.append({
            "agv_id": f"A{k+1:03d}",
            "route": ",".join(clean_path)
        })

    # 5) CSV 저장
    path_df = pd.DataFrame(final_paths)
    output_path = "0926_agv_routes_formatted_2.csv"
    path_df.to_csv(output_path, index=False, encoding="utf-8-sig")
    print(f"📁 AGV 경로가 저장되었습니다: {output_path}")
else:
    print("⚠️ 최적 해를 찾지 못했기 때문에 경로를 저장하지 않습니다.")


NameError: name 'model' is not defined